<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/analise_tags_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dados da máquina virtual

In [1]:
distro_linux = !lsb_release -a
memoria_ram = !free -h
print(distro_linux)
print(memoria_ram)

['No LSB modules are available.', 'Distributor ID:\tUbuntu', 'Description:\tUbuntu 20.04.5 LTS', 'Release:\t20.04', 'Codename:\tfocal']
['              total        used        free      shared  buff/cache   available', 'Mem:           12Gi       547Mi       9.0Gi       1.0Mi       3.1Gi        11Gi', 'Swap:            0B          0B          0B']


# Funções de apoio

In [3]:
import os
import pandas as pd

# dado um diretorio, remove o diretorio
def remove_diretorio_existente(directory):
  if os.path.isdir(directory):
      os.environ['DIRECTORY'] = directory
      print(f"The directory {directory} exists.")
      print(f"Deleting {directory}...")
      comando = !rm -rf $DIRECTORY
      print("Finished!")
  else:
      print(f"The directory {directory} does not exist.")

# dado um diretorio, calcula a quantidade total de arquivos do diretorio
def get_total_arquivos(directory):
  os.environ['DIRECTORY'] = directory
  total_arquivos = !echo "Total de arquivos: " && find $DIRECTORY -type f | wc -l
  total_arquivos = int(total_arquivos[1])
  return total_arquivos

# pega a lista de tags de um diretorio sob gerencia de configuracao do git
def get_lista_tags(directory):
  os.environ['DIRECTORY'] = directory
  lista_tags = !cd $DIRECTORY && git tag
  return lista_tags

# dado um diretorio sob gerencia de configuracao e uma tag, retorna 
# a quantidade de arquivos do diretorio setado para o branch da tag passada
def get_total_arquivos_branch(directory, my_tag):
  os.environ['DIRECTORY'] = directory
  os.environ['MY_TAG'] = my_tag
  comando = !cd $DIRECTORY && git checkout $MY_TAG && echo "Total de arquivos: " && find . -type f | wc -l
  total_arquivos = int(comando[-1])
  return total_arquivos

# dado um diretorio sob gerencia de configuracao e uma tag, retorna uma lista contendo 
# os tipos de arquivos do diretorio
def get_types_of_files_branch(directory, my_tag):
  os.environ['DIRECTORY'] = directory
  os.environ['MY_TAG'] = my_tag
  comando1 = !cd $DIRECTORY && git checkout $MY_TAG
  comando2 = !cd $DIRECTORY && cloc .
  return comando2

# dado um diretorio sob gerencia de configuracao e uma tag, retorna um dataframe contendo 
# os tipos de arquivos do diretorio
def get_texto_cloc_para_df(directory, my_tag):
  os.environ['DIRECTORY'] = directory
  os.environ['EXTENSION'] = '_cloc.txt'
  os.environ['MY_TAG'] = my_tag
  comando1 = !cd $DIRECTORY && git checkout $MY_TAG
  comando2 = !cd $DIRECTORY && cloc . --out=$DIRECTORY$EXTENSION
  my_cloc = directory + '_cloc.txt'
  my_file = directory + '/' + my_cloc
  with open(my_file, 'r') as f:
      lines = f.readlines()[1:]
  with open(my_file, 'w') as f:
      f.writelines(lines)
  # read the text file into a dataframe
  df = pd.read_table(my_file, sep='\s{2,}', engine='python', skiprows=[0, 15, 16], header=None, names=['Language', 'files', 'blank', 'comment', 'code'], index_col='Language')
  # display the dataframe
  df
  return df

# Prepara ambiente

In [4]:
remove_diretorio_existente(directory='cassandra')

The directory cassandra does not exist.


# Análise dos arquivos e tipos de linguagem

In [5]:
!git clone https://github.com/apache/cassandra.git

Cloning into 'cassandra'...
remote: Enumerating objects: 384453, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 384453 (delta 49), reused 135 (delta 47), pack-reused 384299
Receiving objects: 100% (384453/384453), 385.48 MiB | 24.32 MiB/s, done.
Resolving deltas: 100% (223930/223930), done.


In [ ]:
!sudo apt-get install cloc

In [11]:
comando = !cd cassandra && git tag
print(comando[-1])

cassandra-4.1.1


In [7]:
print(f"Total de arquivos: {get_total_arquivos(directory='cassandra')}")

Total de arquivos: 5228


# Lista de tags para análise

lista_tags_cassandra = ['cassandra-3.0.0', 'cassandra-3.1', 'cassandra-3.2', 'cassandra-3.3', 'cassandra-3.4', 'cassandra-3.5', 'cassandra-3.6', 'cassandra-3.7','cassandra-3.8','cassandra-3.9','cassandra-3.10','cassandra-3.11.11']

In [12]:
lista_tags_cassandra = ['cassandra-3.0.0', 'cassandra-3.1', 'cassandra-3.2', 'cassandra-3.3', 'cassandra-3.4', 'cassandra-3.5', 'cassandra-3.6', 'cassandra-3.7','cassandra-3.8','cassandra-3.9','cassandra-3.10','cassandra-3.11.11']

print(f"Qtd de tags: {len(lista_tags_cassandra)}, {lista_tags_cassandra}")

Qtd de tags: 12, ['cassandra-3.0.0', 'cassandra-3.1', 'cassandra-3.2', 'cassandra-3.3', 'cassandra-3.4', 'cassandra-3.5', 'cassandra-3.6', 'cassandra-3.7', 'cassandra-3.8', 'cassandra-3.9', 'cassandra-3.10', 'cassandra-3.11.11']


## Analisa a quantidade total arquivos de cada tag e os tipos de arquivos de cada tag

In [13]:
dict_total_arquivos = {}
dict_dfs = {}
for each_tag in lista_tags_cassandra:
  print(f'Analisando a tag {each_tag}')
  dict_total_arquivos[each_tag] = get_total_arquivos_branch(directory='cassandra', my_tag=each_tag)
  df = get_texto_cloc_para_df(directory='cassandra', my_tag=each_tag)
  dict_dfs[each_tag] = df
  print('Análise concluída!')

Analisando a tag cassandra-3.0.0
Análise concluída!
Analisando a tag cassandra-3.1
Análise concluída!
Analisando a tag cassandra-3.2
Análise concluída!
Analisando a tag cassandra-3.3
Análise concluída!
Analisando a tag cassandra-3.4
Análise concluída!
Analisando a tag cassandra-3.5
Análise concluída!
Analisando a tag cassandra-3.6
Análise concluída!
Analisando a tag cassandra-3.7
Análise concluída!
Analisando a tag cassandra-3.8
Análise concluída!
Analisando a tag cassandra-3.9
Análise concluída!
Analisando a tag cassandra-3.10
Análise concluída!
Analisando a tag cassandra-3.11.11
Análise concluída!


In [19]:
print(dict_total_arquivos['cassandra-3.0.0'])
dict_dfs['cassandra-3.0.0']

2406


,files,blank,comment,code
Language,,,,
Language,files,blank,comment,code
-------------------------------------------------------------------------------,None,None,None,None
Java,1763,58813,67779,316059
Python,25,1811,3007,5713
Ant,3,174,204,1915
ANTLR Grammar,1,198,221,1473
Bourne Shell,34,293,775,1295
PowerShell,4,127,203,808
DOS Batch,19,178,70,683


In [34]:
print(dict_dfs['cassandra-3.0.0']['files']['Java'])
print(dict_dfs['cassandra-3.0.0']['blank']['Java'])
print(dict_dfs['cassandra-3.0.0']['comment']['Java'])
print(dict_dfs['cassandra-3.0.0']['code']['Java'])

1763
58813
67779
316059


In [33]:
l_tags = []
l_qtd_files = []
l_qtd_java_files = []
l_qtd_java_blank = []
l_qtd_java_comment = []
l_qtd_java_loc = []
dict_tags_qtd_files = {}
for k, v in dict_total_arquivos.items():
  l_tags.append(k)
  l_qtd_files.append(v)
  l_qtd_java_files.append(dict_dfs[k]['files']['Java'])
  l_qtd_java_blank.append(dict_dfs[k]['blank']['Java'])
  l_qtd_java_comment.append(dict_dfs[k]['comment']['Java'])
  l_qtd_java_loc.append(dict_dfs[k]['code']['Java'])

dict_tags_qtd_files = {'tag':l_tags, 'qtd_files':l_qtd_files, 'qtd_files_java': l_qtd_java_files, 'qtd_java_blank':l_qtd_java_blank, 'qtd_java_comment': l_qtd_java_comment, 'qtd_java_loc':l_qtd_java_loc}

df_tags_qtd_files = pd.DataFrame(dict_tags_qtd_files)
df_tags_qtd_files

,tag,qtd_files,qtd_files_java,qtd_java_blank,qtd_java_comment,qtd_java_loc
0,cassandra-3.0.0,2406,1763,58813,67779,316059
1,cassandra-3.1,2420,1771,59265,68230,318674
2,cassandra-3.2,2468,1817,60348,69407,325126
3,cassandra-3.3,2477,1826,60759,69751,327066
4,cassandra-3.4,2645,1951,65622,73970,348433
5,cassandra-3.5,2783,1965,66305,74501,351590
6,cassandra-3.6,2829,2006,68023,76400,360937
7,cassandra-3.7,2897,2010,68190,76839,361994
8,cassandra-3.8,3064,2037,69763,80385,369153
9,cassandra-3.9,3064,2037,69763,80385,369153
